<a href="https://colab.research.google.com/github/gkrisp98/SImple-CNN-Classifier/blob/main/CNN_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [ ]:
data_set = torchvision.datasets.FakeData(transform = transforms.Compose([transforms.ToTensor()]))

In [ ]:
len(data_set)

1000

In [ ]:
train_size = int(0.7 * len(data_set))
test_size = len(data_set) - train_size
train_set, test_set = torch.utils.data.random_split(data_set, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_set, batch_size=10, num_workers=1)
num_of_pixels = len(train_set) * 224 * 224 *3

total_sum = 0
for batch in train_loader: total_sum += batch[0].sum()
mean = total_sum / num_of_pixels

sum_of_squared_error = 0
for batch in train_loader: sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

mean, std

(tensor(0.4993), tensor(0.2903))

In [ ]:
test_loader = DataLoader(test_set, batch_size=10, num_workers=1)
num_of_pixels = len(test_set) * 224 * 224 *3

total_sum = 0
for batch in test_loader: total_sum += batch[0].sum()
mean = total_sum / num_of_pixels

sum_of_squared_error = 0
for batch in test_loader: sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

mean, std

(tensor(0.4992), tensor(0.2903))

In [ ]:
train_loader = DataLoader(train_set, batch_size=100, shuffle = True)
data = next(iter(train_loader))
data[0].mean(), data[0].std()

(tensor(0.4992), tensor(0.2903))

In [ ]:
test_loader = DataLoader(test_set, batch_size=32, shuffle = True, num_workers = 2)
data = next(iter(test_loader))
data[0].mean(), data[0].std()

(tensor(0.4994), tensor(0.2902))

In [ ]:
train_set_normal = torchvision.datasets.FakeData(transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)]))

In [ ]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=120*24*24, out_features=500)
    self.out = nn.Linear(in_features=500, out_features=10)
    #self.out = nn.Linear(in_features=100, out_features=10)
  def forward(self, t):
    #hidden conv layer 1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    #hidden conv layer 2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    #hidden conv layer 3
    t = self.conv3(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=3, stride=2)

    #hidden linear layer 1
    t = t.reshape(-1,120*24*24)
    t = self.fc1(t)
    t = F.relu(t)

    #hidden linear layer 2
    #t = self.fc2(t)
    #t = F.relu(t)

    #output layer
    t = self.out(t)
    t = F.softmax(t, dim=1)

    return t

In [ ]:
#TRAINING
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 100)
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(120): #70, for 74% acc
  total_loss = 0
  total_correct = 0
  
  for batch in train_loader:
    images, labels = batch

    preds = network(images) #pass batch
    loss = F.cross_entropy(preds, labels) #calculate loss

    optimizer.zero_grad()
    loss.backward() #calculate gradients
    optimizer.step() #update weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

  print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)

epoch: 0 total_correct: 74 loss: 16.22469735145569
epoch: 1 total_correct: 86 loss: 16.10193419456482
epoch: 2 total_correct: 86 loss: 16.085869073867798
epoch: 3 total_correct: 79 loss: 16.0789053440094
epoch: 4 total_correct: 86 loss: 16.071407794952393
epoch: 5 total_correct: 86 loss: 16.070212364196777
epoch: 6 total_correct: 95 loss: 16.065183877944946
epoch: 7 total_correct: 86 loss: 16.061578273773193
epoch: 8 total_correct: 86 loss: 16.058306455612183
epoch: 9 total_correct: 86 loss: 16.052026748657227
epoch: 10 total_correct: 95 loss: 16.046431064605713
epoch: 11 total_correct: 103 loss: 16.03572726249695
epoch: 12 total_correct: 107 loss: 16.02239418029785
epoch: 13 total_correct: 118 loss: 16.003053665161133
epoch: 14 total_correct: 111 loss: 15.973464012145996
epoch: 15 total_correct: 138 loss: 15.92969012260437
epoch: 16 total_correct: 127 loss: 15.923389911651611
epoch: 17 total_correct: 174 loss: 15.796257495880127
epoch: 18 total_correct: 111 loss: 15.912986278533936
ep

In [ ]:
total_correct / len(train_set)

0.74

In [ ]:
Preds = network(images)

In [ ]:
_, predicteD = torch.max(preds, 1)

In [ ]:
predicteD

tensor([3, 5, 5, 4, 4, 3, 5, 5, 5, 5])

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        preds = network(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

Accuracy: 8 %
